In [0]:
!pip install fasttext

     |████████████████████████████████| 71kB 4.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3020410 sha256=01497438a9c5073935f4adbdf3eb00051907b6008827e57274a3157b1f408e8d
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Models/cc.lv.300.bin'
import warnings
warnings.filterwarnings('ignore')

In [0]:
import fasttext
ft = fasttext.load_model('cc.lv.300.bin')

In [0]:
max_words = len(ft.get_words())
print('Vārdnīcas izmērs:',max_words)

Vārdnīcas izmērs: 1641469


In [0]:
# Vārdu piemēri - tajos atrodams pilnīgi viss
words = ft.get_words()
for i in range(30):
  print(words[100*i:100*i+100])

[',', '.', '</s>', 'un', '"', ')', '(', ':', 'ir', 'ar', 'par', 'no', '-', "'", 'arī', 'uz', 'kas', 'vai', '/', 'kā', '—', 'ka', '”', '–', 'gada', 'gadā', 'bija', '_', 'to', 'bet', ';', 'Latvijas', 'līdz', '?', '!', 'pēc', 'var', '}', '“', '1', 'lai', 'tika', 'nav', 'tā', '...', '„', 'tiek', 'tikai', 'gan', '2', 'tas', 'laikā', '€', 'jau', 'pie', 'ko', '+', '%', 'valsts', 'jo', 'savu', 'ja', '3', 'Rīgas', 'tās', 'šo', '|', 'kad', 'vairāk', 'tad', '10', 'Latvijā', 'Lai', 'kur', 'ļoti', 'vietā', 'darba', '4', 'viņš', '#', 'gadu', '*', 'vēl', 'pa', 'Pēc', 'bez', '5', 'atrodas', 'ne', 'visu', 'Ja', 'viņa', 'Eiropas', 'pret', 'tam', 'būs', 'Kategorija', 'viesnīcas', 'darbu', 'savā']
['0', 'Par', 'kuru', 'būt', 'Rīga', 'No', '»', 'vārds', 'iela', 'latviešu', 'pat', 'skaits', 'km', 'ASV', 'Tā', 'spēles', '18', 'kurā', 'mūsu', '6', 'novada', 'kopā', 'nekā', 'daudz', 'Tas', '1.', 'es', 'pilsētas', 'taču', '00', 'vieta', 'kurš', '12', 'kuras', 'viens', 'pirms', 'pasaules', 'numurs', 'viena', 'bū

In [0]:
dataset = pd.read_csv('papildinato_jautajumu_kopa.csv', index_col=0)
dataset.head(1)

,Question,labels
0,"1. septembrī var noslēgt DV līgumu? Un ja var,...",2


In [0]:
dataset['labels'].value_counts()

1    23025
3      122
2      122
0      122
Name: labels, dtype: int64

In [0]:
questions = dataset['Question'].values.astype('str')
labels    = dataset['labels']

In [0]:
exec_start = datetime.datetime.now()
print(exec_start)
encoded_questions = []
for question in questions:
  sentence_vector = ft.get_sentence_vector(question)        # Iegūstam teikuma vektoru
  encoded_questions.append(sentence_vector)

exec_end = datetime.datetime.now()
exec_elapsed = exec_end - exec_start
print(exec_end)
print('Total time elapsed:', exec_elapsed)

2020-05-10 08:26:44.563709
2020-05-10 08:26:51.433679
Total time elapsed: 0:00:06.869970


In [0]:
fasttext_df = pd.DataFrame(encoded_questions)
fasttext_df['labels'] = labels
print(fasttext_df.shape)
fasttext_df.to_csv('Fasttext_embeddings.csv')

(23391, 301)


In [0]:
features = fasttext_df.iloc[:,:300]
target = fasttext_df.iloc[:,300:]
print(features.shape, target.shape)

(23391, 300) (23391, 1)


<h2>Izveidoto reprezentāciju darbības pārbaude</h2>


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state = 42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(18712, 300) (18712, 1)
(4679, 300) (4679, 1)


In [0]:
from keras import layers
from keras import models
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from keras.preprocessing import text
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [0]:
# Funkcija neironu tīkla atbildes varbūtību masīva pārveidošanai par atbildes klasi
def transformResults(results):
    mod_results = [];
    print("Initial shape: ", results.shape)
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)
    print("Modified shape: ", mod_results.shape)    
    return mod_results 

In [0]:
y_train2 = y_train
y_test2 = y_test

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(300,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss=['sparse_categorical_crossentropy'],
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])
batch_size = 64
epochs = 5
history = model.fit(x_train, y_train2,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
y_pred = model.predict(x_test)                                                  # Prasam modelim paredzēt testa datiem kategorijas
y_pred2 = transformResults(y_pred)                                              # Pārveidojam rezultātus

print(confusion_matrix(y_test2, y_pred2))
print(classification_report(y_test2, y_pred2, target_names=class_names))

Train on 16840 samples, validate on 1872 samples
Epoch 1/5
16840/16840 [==============================] - 1s 70us/step - loss: 0.1461 - sparse_categorical_accuracy: 0.9836 - val_loss: 0.0658 - val_sparse_categorical_accuracy: 0.9850
Epoch 2/5
16840/16840 [==============================] - 1s 50us/step - loss: 0.0623 - sparse_categorical_accuracy: 0.9837 - val_loss: 0.0460 - val_sparse_categorical_accuracy: 0.9861
Epoch 3/5
16840/16840 [==============================] - 1s 51us/step - loss: 0.0446 - sparse_categorical_accuracy: 0.9875 - val_loss: 0.0321 - val_sparse_categorical_accuracy: 0.9899
Epoch 4/5
16840/16840 [==============================] - 1s 50us/step - loss: 0.0328 - sparse_categorical_accuracy: 0.9909 - val_loss: 0.0292 - val_sparse_categorical_accuracy: 0.9899
Epoch 5/5
16840/16840 [==============================] - 1s 50us/step - loss: 0.0285 - sparse_categorical_accuracy: 0.9919 - val_loss: 0.0252 - val_sparse_categorical_accuracy: 0.9899
Initial shape:  (4679, 4)
Modif

In [0]:
# Notestējam izveidoto modeli
test_sentence = ['Kāds ir bibliotēkas darba laiks?',
                 'Kas jādara, lai paņemtu akadēmisko gadu?',
                 'Vai PPMFā ir ēdnīca?',
                 'Kādi dokumenti jāiesniedz, parakstot studiju līgumu?']
test_encoded = []
for sentence in test_sentence:
  test_encoded.append(ft.get_sentence_vector(sentence))
print(len(test_encoded), len(test_encoded[0]))

counter = 0
for encoded in test_encoded:                              # Katram no iekodētajiem jautājumiem
  reshaped = np.reshape(encoded, (1,300))                 # Pārveidojam jautājumu atbilstoši modeļa ieejas formātam
  output = model.predict(reshaped)                        # Prasam modelim paredzēt klašu varbūtības
  print('Jautājums:',test_sentence[counter])
 
  label = np.argmax(output)
  if label == 0:
    print('Kategorija: Akadēmisks')
  elif label == 1:
    print('Kategorija: Citi jautājumi')
  elif label == 2:
    print('Kategorija: Darba laiks')
  elif label == 3:
    print('Kategorija: Dokumenti')
  counter = counter + 1
  print('Klašu varbūtības:', output, '\n')

4 300
Jautājums: Kāds ir bibliotēkas darba laiks?
Kategorija: Darba laiks
Klašu varbūtības: [[5.9658947e-04 3.1820547e-02 9.6726203e-01 3.2081190e-04]] 

Jautājums: Kas jādara, lai paņemtu akadēmisko gadu?
Kategorija: Akadēmisks
Klašu varbūtības: [[9.0762532e-01 6.9512405e-02 5.1609270e-04 2.2346172e-02]] 

Jautājums: Vai PPMFā ir ēdnīca?
Kategorija: Citi jautājumi
Klašu varbūtības: [[1.8308059e-06 9.9999058e-01 8.7533863e-07 6.8171153e-06]] 

Jautājums: Kādi dokumenti jāiesniedz, parakstot studiju līgumu?
Kategorija: Dokumenti
Klašu varbūtības: [[2.9975138e-04 1.8806223e-02 4.2896165e-05 9.8085117e-01]] 

